# My summary

## Deep dive into the building blocks of neural networks

As we learned in the previous chapter, training a deep learning algorithm requires the following steps:
1. Building a data pipeline
2. Building a network architecture
3. Evaluating the architecture using a loss function
4. Optimizing the network architecture weights using an optimization algorithm

### Non-linear activations

Non-linear activations are functions that take inputs and then apply a mathematical transformation and produce an output. There are several non-linear operations that we come across in practice. We will go through some of the popular non-linear activation functions.

#### Sigmoid

The sigmoid function intuitively takes a real-valued number and outputs a number in a range between zero and one. For a large negative number, it returns close to zero and, for a large positive number, it returns close to one. The following plot represents different sigmoid function outputs:

![](images/Sigmoid.jpg)

The sigmoid function has been historically used across different architectures, but in recent times it has gone out of popularity as it has one major drawback. When the output of the sigmoid function is close to zero or one, the gradients for the layers before the sigmoid function are close to zero and, hence, the learnable parameters of the previous layer get gradients close to zero and the weights do not get adjusted often, resulting in dead neurons.

#### Tanh

The tanh non-linearity function squashes a real-valued number in the range of -1 and 1. The tanh also faces the same issue of saturating gradients when tanh outputs extreme values close to -1 and 1. However, it is preferred to sigmoid, as the output of tanh is zero centered:

![](images/Tanh.jpg)

#### ReLU

ReLU has become more popular in the recent years; we can find either its usage or one of its variants' usages in almost any modern architecture. In simple words, ReLU squashes any input that is negative to zero and leaves positive numbers as they are. We can visualize the ReLU function as follows:

![](images/ReLU.jpg)

Some of the pros and cons of using ReLU are as follows:
- It helps the optimizer in finding the right set of weights sooner. More technically it makes the convergence of stochastic gradient descent faster.
- It is computationally inexpensive, as we are just thresholding and not calculating anything like we did for the sigmoid and tangent functions.
- ReLU has one disadvantage; when a large gradient passes through it during the backward propagation, they often become non-responsive; these are called dead neutrons, which can be controlled by carefully choosing the learning rate.

#### Leaky ReLU

Leaky ReLU is an attempt to solve a dying problem where, instead of saturating to zero, we saturate to a very small number such as 0.001. For some use cases, this activation function provides a superior  performance to others, but it is not consistent.

### PyTorch non-linear activations

PyTorch has most of the common non-linear activation functions implemented for us already and it can be used like any other layer. Let's see a quick example of how to use the `ReLu` function in PyTorch:

In [13]:
from torch import Tensor, nn
from torch.autograd import Variable

In [15]:
sample_data = Variable(torch.Tensor([1, 2, -1, -1]))
myReLU = nn.ReLU()
print(myReLU(sample_data))

tensor([1., 2., 0., 0.])


### The PyTorch way of building deep learning algorithms

All the networks in PyTorch are implemented as classes, subclassing a PyTorch class called `nn.Module`, and should implement `__init__` and `forward` methods. Inside the `__init__` function, we initialize any layers, such as the `linear` layer. In the `forward` method, we pass our input data into the layers that we initialized in our `__init__` method and return our final output. The non-linear functions are often directly used in the `forward` function and some use it in the `__init__` method too. The following code snippet shows how a deep learning architecture is implemented in PyTorch:

In [16]:
class MyFirstNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyFirstNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
    def __forward__(self, input):
        out = self.layer1(input)
        out = nn.ReLU(out)
        out = self.layer2(out)
        return out

### Model architecture for different machine learning problems

The kind of problem we are solving will decide mostly what layers we will use, starting from a linear layer to **Long Short-Term Memory (LSTM)** for sequential data. Based on the type of the problem you are trying to solve, your last layer is determined. There are three problems that we generally solve using any machine learning or deep learning algorithms. Let's look at what the last layer would look like:

1. For a regression problem, such as predicting the price of a t-shirt to sell, we would use the last layer as a linear layer with an output of one, which outputs a continuous value.
2. For classifying a given image as t-shirt or shirt, you would use a sigmoid activation function, as it outputs values either closer to one or zero, which is generally called a **binary classification problem**.
3. For a multi-class classification, where we have to classify whether a given image is a t-shirt, jeans, shirt, or dress, we would use a softmax layer at the end our network. Let's try to understand intuitively what softmax does without going into the math of it. It takes inputs from the previous linear layer, for example, and outputs the probabilities for a given number of examples. In our example, it would be trained to predict four probabilities for each type of image. Remember, all these probabilities always add up to one.

### Loss functions

Once we have defined our network architecture, we are left with two important steps. One is calculating how good our network is at performing a particular task of regression, classification, and the next is optimizing the weight.

The optimizer (gradient descent) generally accepts a scalar value, so our `loss` function should generate a scalar value that has to be minimized during our training. Certain use cases, such as predicting where an obstacle is on the road and classifying it to a pedestrian or not, would require two or more loss functions. Even in such scenarios, we need to combine the losses to a single scalar for the optimizer to minimize.

PyTorch provides several implementations of commonly used `loss` functions. Let's take a look at the `loss` functions used for regression and classification.

|Loss funtion  |Used                                  |
| :---         |                                          ---:|
|L1Loss            |Mostly used as a regularizer.                 |
|MSELoss          |Used as loss function for regression problems.|
|CrossEntropyLoss |Used for binary and multi-class classification problems.|
|NLLLoss |Used for classification problems and allows us to use specific weights to handle imbalanced datasets.|
|NLLLoss2d |Used for pixel-wise classification, mostly for problems related to image segmentation.|

The commonly used `loss` function for regression problems is mean square error (MSE). We can use the `loss` function implemented in PyTorch, as follows:

In [22]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True)
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

For classification, we use a cross-entropy loss. Before looking at the math for cross-entropy, let's understand what a cross-entropy loss does. It calculates the loss of a classification network predicting the probabilities, which should sum up to one, like our softmax layer. A cross-entropy loss increases when the predicted probability diverges from the correct probability.

To use a cross-entropy loss in a classification problem, we really do not need to be worried about what happens insideball we have to remember is that, the loss will be high when our predictions are bad and low when predictions are good. PyTorch provides us with an implementation of the `loss` , which we can use, as follows:

In [24]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True)
target = Variable(torch.LongTensor(3).random_(5))
output = loss(input, target)
output.backward()

### Optimizing network architecture

Once we have calculated the loss of our network, we will optimize the weights to reduce the loss and thus improving the accuracy of the algorithm. For the sake of simplicity, let's see these optimizers as black boxes that take loss functions and all the learnable parameters
and move them slightly to improve our performances. PyTorch provides most of the commonly used optimizers required in deep learning.

Some of the optimizers that PyTorch provides are as follows:
- ADADELTA
- Adagrad
- Adam
- SparseAdam
- Adamax
- ASGD
- LBFGS
- RMSProp
- Rprop
- SGD

In `optimizer = optim.SGD(model.parameters(), lr=0.01)`, we created an `SGD` optimizer that takes all the learnable parameters of your network as the first argument and a learning rate that determines what ratio of change can be made to the learnable parameter.

Once you create an optimizer object, we need to call `zero_grad()` inside our loop, as the parameters will accumulate the gradients created during the `optimizer` call. 